In [1]:
import os, gc, time
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
username = os.getenv('USER')
import dask.array as da
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
import dask
from collections import OrderedDict
dask.config.set(**{'array.slicing.split_large_chunks': False})
# dask.config.set({"distributed.workers.memory.target": 4})

from dscim import ProWaiter
from dscim.menu.simple_storage import StackedDamages, EconVars, Climate
from dscim.menu.risk_aversion import RiskAversionRecipe
from dscim.menu.baseline import Baseline
from dscim.menu.equity import EquityRecipe
from itertools import product, chain
import itertools
# import seaborn as sns
from dscim.fair.utils import modeler
from dscim.fair.estimations import c_equivalence
from functools import reduce
from pathlib import Path

import seaborn as sns
from dscim.fair.estimations import compute_damages

In [2]:
discount_types = ["euler_gwr", "constant", "euler_ramsey"] # , "naive_gwr", "naive_ramsey"]
menu_options = ["risk_aversion", "adding_up", "equity"]
pulse_years = [2020]
# pulse_years = list(range(2025, 2051, 5))
weitzman_values = [x / 10.0 for x in range(1, 11, 1)] + [0.25, 0.01, 0.001, 0.0001]
global_cons = False
masks = ["unmasked",
         "truncate_at_ecs995symmetric_passing_mask",
        "truncate_at_ecs990symmetric_passing_mask",
        "truncate_at_ecs950symmetric_passing_mask",
        "truncate_at_ecs830symmetric_passing_mask",
        "truncate_at_ecs750symmetric_passing_mask",
        ]

# run all

In [3]:
sectors = ["integration_CAMEL_clipped"] # "integration_AMEL_clipped", "coastal", 
AR_list = [5,6]
combos = [(s, p, d, m, ar, ma) 
          for p in pulse_years
          for s in sectors 
          for d in discount_types 
          for m in menu_options
          for ar in AR_list
          for ma in masks
         ]

In [ ]:
n_combos = len(combos)

while n_combos > 0: 
    combo = combos[0]
    print("=========================================================")
    print(combo)
    sector = combo[0]
    pulse_year = combo[1]
    discount_type = combo[2]
    menu_option = combo[3]
    AR = combo[4]
    mask = combo[5]

    w = ProWaiter(path_to_config=f'/home/{username}/repos/integration/configs/epa_tool_config-histclim_AR{AR}.yaml')
    save_path=f'/mnt/CIL_integration/menu_results_AR{AR}_bc39_integration/{sector}/{pulse_year}/{mask}/'

    if mask=='unmasked':
        mask = None

    kwargs = {'discounting_type' : discount_type,
              'sector': sector,
              'ce_path' : f'/shares/gcp/integration/CE_library_epa_vsl_bc39/{sector}/',
              'save_path' : save_path,
              'weitzman_parameter' : weitzman_values,
              'pulse_year' : pulse_year,
              'ecs_mask_name' : mask
             }

    if "CAMEL" in sector:
        kwargs.update({
            'damage_function_path' : f'/mnt/CIL_integration/damage_function_library/damage_function_library_integration/{sector}'})

    menu_item = w.menu_factory(menu_key=menu_option,
                                           sector=sector,
                                           kwargs=kwargs
                                          )
    menu_item.order_plate('scc')
    if "CAMEL" not in sector:
        menu_item.damage_function_fit
        
    if global_cons == True:
        menu_item.global_consumption_no_pulse.to_netcdf(
            f"{save_path}/{menu_option}_{discount_type}_eta{menu_item.eta}_rho{menu_item.rho}_global_consumption_no_pulse.nc4")

    combos.remove(combo)
    n_combos = len(combos)
    print(f"remaining combos: {n_combos}")


('integration_CAMEL_clipped', 2020, 'euler_gwr', 'risk_aversion', 5, 'unmasked')

 Executing Running risk_aversion class with the following parameters: 
Sector: integration_CAMEL_clipped 
Path to damages: None 
Menu recipe: risk_aversion 
Damages delta: summed_delta 
Discount level: global 
Discount type: euler_gwr 
Damage function path: /mnt/CIL_integration/damage_function_library/damage_function_library_integration/integration_CAMEL_clipped 


Processing damage functions ...
You have passed a path to damage functions. Loading damage function coefficients...
Saving /mnt/CIL_integration/menu_results_AR5_bc39_integration/integration_CAMEL_clipped/2020/unmasked/risk_aversion_euler_gwr_eta2.0_rho0.0_damage_function_coefficients.nc4
Processing SCC calculation ...
You have passed a path to damage functions. Loading damage function coefficients...
Saving /mnt/CIL_integration/menu_results_AR5_bc39_integration/integration_CAMEL_clipped/2020/unmasked/risk_aversion_euler_gwr_eta2.0_rho0.0_damage

/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


Skipped saving.


/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/xarray/core/nanops.py:148: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/xarray/core/nanops.py:148: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


Skipped saving.
continuous discounting
continuous discounting
You have passed a path to damage functions. Loading damage function coefficients...
Saving /mnt/CIL_integration/menu_results_AR5_bc39_integration/integration_CAMEL_clipped/2020/unmasked/risk_aversion_euler_gwr_eta2.0_rho0.0_damage_function_coefficients.nc4
continuous discounting
Saving /mnt/CIL_integration/menu_results_AR5_bc39_integration/integration_CAMEL_clipped/2020/unmasked/risk_aversion_euler_gwr_eta2.0_rho0.0_discount_factors.nc4


...

/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


Skipped saving.
Skipped saving.
continuous discounting
continuous discounting
You have passed a path to damage functions. Loading damage function coefficients...
Saving /mnt/CIL_integration/menu_results_AR6_bc39_integration/integration_CAMEL_clipped/2020/truncate_at_ecs995symmetric_passing_mask/risk_aversion_euler_gwr_eta2.0_rho0.0_damage_function_coefficients.nc4
continuous discounting
Saving /mnt/CIL_integration/menu_results_AR6_bc39_integration/integration_CAMEL_clipped/2020/truncate_at_ecs995symmetric_passing_mask/risk_aversion_euler_gwr_eta2.0_rho0.0_discount_factors.nc4


/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/minic

Skipped saving.
You have passed a path to damage functions. Loading damage function coefficients...
Saving /mnt/CIL_integration/menu_results_AR6_bc39_integration/integration_CAMEL_clipped/2020/truncate_at_ecs995symmetric_passing_mask/risk_aversion_euler_gwr_eta2.0_rho0.0_damage_function_coefficients.nc4
Skipped saving.
You have passed a path to damage functions. Loading damage function coefficients...
Saving /mnt/CIL_integration/menu_results_AR6_bc39_integration/integration_CAMEL_clipped/2020/truncate_at_ecs995symmetric_passing_mask/risk_aversion_euler_gwr_eta2.0_rho0.0_damage_function_coefficients.nc4
You have passed a path to damage functions. Loading damage function coefficients...
Saving /mnt/CIL_integration/menu_results_AR6_bc39_integration/integration_CAMEL_clipped/2020/truncate_at_ecs995symmetric_passing_mask/risk_aversion_euler_gwr_eta2.0_rho0.0_damage_function_coefficients.nc4
Saving /mnt/CIL_integration/menu_results_AR6_bc39_integration/integration_CAMEL_clipped/2020/truncate

/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/minic